In [1]:
import json
import time
import pickle

import requests
import pandas as pd

url_main = 'https://api.hh.ru/vacancies'
param = "?"
hosts = ["host=hh.ua", "host=hh.ru", "host=career.ru", "host=jobs.tut.by",
        "host=jobs.day.az", "host=hh.uz", "host=hh.kz", "host=headhunter.ge",
       "host=headhunter.kg"]
an = "&"
per_p = "per_page="
page = "page="
url =  'https://api.hh.ru/vacancies?host=hh.ua&per_page=100&page='

In [2]:
r = requests.get('https://api.hh.ru/vacancies?host=hh.ua&per_page=100&page=0')
print(r.status_code)
#print(r.text)
#vacancies = json.loads(r.text)
#print(json.loads(r.text)["items"][0])
#print()
#print(json.loads(r.text)["items"][0]["snippet"])
len(json.loads(r.text)["items"])

200


100

In [3]:

def load_vacancies_ids(url, hosts):
    vacancies = []
    vac_ids = []
    param = "?"
    an = "&"
    per_page = "per_page=100"
    page = "page="
    for host in hosts:
        for i in range(20):
            url_mod = url + param + host
            url_mod = url_mod + an + per_page + an + page + str(i)
            r = requests.get(url_mod)
            vac = json.loads(r.text)
            vacancies = vacancies + vac["items"]
            vac_ids = vac_ids + [i["id"] for i in vac["items"]]
    return vac_ids, vacancies



In [4]:
%%time
vac_ids, vacancies = load_vacancies_ids(url_main, hosts)

Wall time: 1min 52s


In [5]:
len(vac_ids)

18000

In [6]:
ids_set = set(vac_ids)

In [7]:
len(ids_set)

3616

In [8]:
list(ids_set)[100]

'28844566'

In [9]:
with open("headHunter_data/hh_ids.dat", "rb") as inf:
    ids2_list = pickle.load(inf)
    
with open("headHunter_data/hh_vacancies.dat", "rb") as inf:
    vacancies2 = pickle.load(inf)
    
with open("headHunter_data/hh_vacancies_ext.dat", "rb") as inf:
    vacancies_ext2 = pickle.load(inf)

In [10]:
print(len(vacancies_ext2))
print(len(ids2_list))
#print(len(ids2_list + list(ids_set)))

ids_all = set(ids2_list + list(ids_set))

print(len(ids_all))

25521
24980
25378


In [11]:
len(ids_set.intersection(set(ids2_list)))

3218

In [12]:
unic_ids = ids_set.difference(ids_set.intersection(set(ids2_list)))
print(len(unic_ids))

398


In [13]:
vacancies = vacancies2 + vacancies
print(len(vacancies))

144000


In [14]:
ids_all = set(ids2_list + list(unic_ids))
print(len(ids_all))

24980


In [15]:

def load_vacancies_extended(url, ids):
    vacancies_ext = []
    count = 0
    for i in ids:
        url_mod = url + "/" + str(i)
        r = requests.get(url_mod)
        vacancy = json.loads(r.text)
        vacancies_ext.append(vacancy)
        count += 1
        
        if count % 500 == 0:
            print(count)
    return vacancies_ext


In [16]:
%%time
vacancies_ext = load_vacancies_extended(url_main, list(unic_ids))

500
1000
1500
2000
2500
Wall time: 12min 34s


In [17]:
print(len(vacancies_ext))
print(len(vacancies_ext2))

2717
22804


In [18]:
vacancies_ext = vacancies_ext2 + vacancies_ext
print(len(vacancies_ext))

25521


### Save loaded new vacancies and ald

In [19]:
with open("hh_ids.dat", "wb") as ouf:
    pickle.dump(list(ids_all), ouf)
    
with open("hh_vacancies.dat", "wb") as ouf:
    pickle.dump(vacancies, ouf)

with open("hh_vacancies_ext.dat", "wb") as ouf:
    pickle.dump(list(vacancies_ext), ouf)

In [ ]:

with open("hh_ids.json", "w") as ouf:
    json.dump(list(ids_all), ouf, ensure_ascii=False)
    
with open("hh_vacancies.json", "w") as ouf:
    json.dump(vacancies, ouf, ensure_ascii=False)
    
with open("hh_vacancies_ext.json", "w") as ouf:
    json.dump(vacancies_ext, ouf, ensure_ascii=False)
    

### Check saved data

In [20]:
with open("hh_ids.dat", "rb") as inf:
    t = pickle.load(inf)
print("uniq vacancies ids number =", len(t))

with open("hh_vacancies.dat", "rb") as inf:
    t = pickle.load(inf)
print("num total loaded vac description =", len(t))

with open("hh_vacancies_ext.dat", "rb") as inf:
    t = pickle.load(inf)
print("num loaded full vac =", len(t))    

uniq vacancies ids number = 24980
num total loaded vac description = 144000
num loaded full vac = 25521


In [3]:
r = requests.get('https://drive.google.com/open?id=1-XZVxCYUdARwi8BZYOsV_M3uKPrXaVYf')
print(r.status_code)
print(r.text)

200
<!DOCTYPE html><html><head><meta name="google" content="notranslate"><meta http-equiv="X-UA-Compatible" content="IE=edge;"><style>@font-face{font-family:'Roboto';font-style:italic;font-weight:400;src:local('Roboto Italic'),local('Roboto-Italic'),url(//fonts.gstatic.com/s/roboto/v18/KFOkCnqEu92Fr1Mu51xMIzc.ttf)format('truetype');}@font-face{font-family:'Roboto';font-style:normal;font-weight:300;src:local('Roboto Light'),local('Roboto-Light'),url(//fonts.gstatic.com/s/roboto/v18/KFOlCnqEu92Fr1MmSU5fABc9.ttf)format('truetype');}@font-face{font-family:'Roboto';font-style:normal;font-weight:400;src:local('Roboto Regular'),local('Roboto-Regular'),url(//fonts.gstatic.com/s/roboto/v18/KFOmCnqEu92Fr1Mu5mxP.ttf)format('truetype');}@font-face{font-family:'Roboto';font-style:normal;font-weight:700;src:local('Roboto Bold'),local('Roboto-Bold'),url(//fonts.gstatic.com/s/roboto/v18/KFOlCnqEu92Fr1MmWUlfABc9.ttf)format('truetype');}</style><title>encoder_model.json - Google Диск</title><meta proper